# E-76 Obligated Federal Funds Data Exploration
### From the Division of Local Assistance 

E-76's or "Authorizations to Proceed" is a federal authorization of funds to establish the reimbursement date for a phase of work 

#### Data Sources: 
- https://dot.ca.gov/programs/local-assistance/reports/e-76-obligated

In this notebook, we will be looking to see what types of data is in the spreadsheet as well as begin to clean it for futher analysis


In [2]:
##one time load 
#!pip install plotly-express
#!pip install contextily
#!pip install openpyxl
#!pip install geopandas


In [1]:
import pandas as pd
import geopandas as gpd
from siuba import _, mutate, count, filter, group_by, ungroup
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import contextily as ctx

## Reading Excel as a data dictionary
Can use this approach, but recommend the next with second one. 

In [7]:
#obligated = pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/obligated-projects.xlsx', sheet_name=['2014-2018', '2019', '2020', '2021'])

In [3]:
#print(obligated)

In [4]:
#print(type(obligated))

In [5]:
#len(obligated)

In [6]:
#print(type(obligated['2020']))

In [7]:
#obligated['2020'].info()

In [8]:
#obligated['2021'].info()

In [9]:
#obligated['2019'].info()

In [10]:
#obligated['2014-2018'].info()

In [11]:
#df1 = obligated['2014-2018']
#df2 = obligated['2019']
#df3 = obligated['2020']
#df4 = obligated['2021']

In [12]:
#pd_dct = {"pre2018": df1, "2019": df2, "2020":df3, "2021":df4}

In [13]:
#mdf = pd.concat(pd_dct, axis=0).reset_index(level=0).rename({'level_0':'key'}, axis=1)

In [14]:
#mdf

## Reading Excel with Concat

In [2]:
#attempt 2
## code source: https://pbpython.com/pandas-excel-tabs.html
odf = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/obligated-projects.xlsx', sheet_name=None), ignore_index=True)



/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:71: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
odf.head()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,...,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,...,9.0,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,NaN,NaN
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,...,7.0,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,NaN
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,...,6.0,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,NaN
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,...,0.0,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,NaN
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,...,0.0,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,NaN


In [4]:
odf.tail()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,...,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28
20016,Obligated,BRLO,5955(095),Orange County,2021-08-20 00:00:00,2021-08-27 00:00:00,2021-08-31 00:00:00,2021-09-01 00:00:00,2021-09-17 00:00:00,82000.0,...,16.0,ORA020501,"Silverado Canyon Road Over Silverado Creek, Ap...",Bridge Replacement (tc).,2,2021-08-20 00:00:00,2021-08-27 00:00:00,SCAG,NaT,NaN
20017,Obligated,HSIPL,5073(093),Orange,2021-08-17 00:00:00,2021-08-27 00:00:00,2021-08-30 00:00:00,2021-08-30 00:00:00,2021-09-08 00:00:00,21000.0,...,9.0,SCAG015,Intersection Of Glassell St. & Palmyra Ave.,Install A New 2-phase Traffic Signal,2,2021-07-28 00:00:00,2021-08-26 00:00:00,SCAG,NaT,NaN
20018,Obligated,HSIPL,5073(078),Orange,2021-08-18 00:00:00,2021-08-27 00:00:00,2021-09-01 00:00:00,2021-09-01 00:00:00,2021-09-17 00:00:00,0.0,...,16.0,SCAG015,Intersection Of Glassell Street And Walnut Avenue,Protected Left-turn Signal Phasing,4,2021-08-13 00:00:00,2021-08-27 00:00:00,SCAG,NaT,NaN
20019,Obligated,HSIPL,5073(080),Orange,2021-08-18 00:00:00,2021-08-27 00:00:00,2021-09-01 00:00:00,2021-09-01 00:00:00,2021-09-17 00:00:00,0.0,...,16.0,SCAG015,Intersection Of Chapman Avenue And James Street,Install Protected Left Turn Signal Phasing,4,2021-08-12 00:00:00,2021-08-27 00:00:00,SCAG,NaT,NaN
20020,Obligated,CMSTPLN,6071(129),Orange County Transportation Authority,2021-08-23 00:00:00,2021-08-24 00:00:00,2021-08-31 00:00:00,2021-08-31 00:00:00,2021-09-09 00:00:00,8000000.0,...,9.0,ORA100511,State Route 55 From Interstate 405 To Intersta...,Add One Mixed-flow And High-occupancy Lane In ...,6,2021-06-01 00:00:00,2021-08-22 00:00:00,SCAG,NaT,NaN


In [5]:
print(len(odf))

20021


In [6]:
odf.sample(10)

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,...,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28
3490,Obligated,DEM10L,5209(008),Beaumont,2017-07-31,2017-08-07,2017-08-30,2017-09-13,2017-09-15,13443872.00,...,2.0,RIV050535,Intersection Of State Route 60 And Potrero Bou...,"New Overcrossing For Future Interchange, Phase 1",1,2017-07-31,2017-08-07,SCAG,NaN,NaN
16747,Obligated,BHLS,5405(075),Simi Valley,2019-11-12 00:00:00,2020-06-10 00:00:00,2020-06-11 00:00:00,2020-06-11 00:00:00,2020-06-16 00:00:00,0.00,...,5.0,VENLS07,"Bernard Street Over Las Lljas Creek, 0.5 Miles...",Widen Existing 2 Lane Bridge With A 2 Lane Bri...,2,2020-06-09 00:00:00,2020-06-10 00:00:00,SCAG,NaN,NaN
5028,Obligated,BRLS,5002(134),Sacramento,2017-01-25,2017-01-25,2017-01-31,2017-02-06,2017-02-08,70824.00,...,2.0,GROUP23,Rio Linda Blvd. At Magpie Creek ~1.0 Mile N. O...,Bridge Rehabilitation,4,NaN,NaN,SACOG,NaN,NaN
11980,Obligated,BRLS,5906(119),Shasta County,2014-04-23,2014-04-23,2014-04-30,2014-05-02,2014-05-02,66397.00,...,0.0,BRDGE-LUM,On Gas Point Rd At No Name Ditch Br.# 06c0183,Bridge Rehabilitation,1,NaN,NaN,STPA,NaN,NaN
9265,Obligated,BPMPL,5085(034),Merced,2015-04-28,2015-04-28,2015-04-30,2015-05-08,2015-05-12,35412.00,...,4.0,CT004,Several Locations (bridges 39c0028 & 39c0041),Bridge Preventive Maintenance,3,NaN,NaN,SCAG,NaN,NaN
3357,Obligated,CML,5208(110),Clovis,2017-10-04,2017-10-04,2017-10-04,2017-10-04,2017-10-16,-0.03,...,12.0,FRE090104,Shaw Avenue And Dewolf Avenue Intersection,Installation Of Traffic Signal (tc),6,NaN,NaN,CFCG,NaN,NaN
19409,Obligated,RPSTPLE,5953(685),Los Angeles County,2021-06-14 00:00:00,2021-06-14 00:00:00,2021-06-16 00:00:00,2021-06-24 00:00:00,2021-06-25 00:00:00,-145559.96,...,1.0,LAF5514,Vermont Ave - From Manchester Blvd To El Segun...,Class Ii Bike Lane Construction(3 Miles),6,2021-06-10 00:00:00,2021-06-10 00:00:00,SCAG,NaT,NaN
5744,Obligated,HSIPL,5063(171),Santa Ana,2016-09-15,2016-09-15,2016-09-15,2016-09-15,2016-09-20,54000.00,...,5.0,SCAG015,Both Directions On Hazard Avenue At The Inters...,Install Left-turn Phasing,1,NaN,NaN,SCAG,NaN,NaN
9138,Obligated,CML,5950(394),Kern County,2015-05-12,2015-05-12,2015-05-28,2015-05-29,2015-05-31,1445694.00,...,2.0,KER140509,Holloway Road: From Sr46 To Twisselman Rd.,Construct 6' Shoulder W'2' Dirt Shldr Backing,1,NaN,NaN,KCOG,NaN,NaN
8885,Obligated,BRLO,5915(056),Colusa County,2015-06-17,2015-06-17,2015-06-17,2015-06-18,2015-06-24,-2223.71,...,6.0,NaN,Leesville-lodoga Road At Ltl Indian Creek,Bridge Replacement Br#15c0131,2,NaN,NaN,NON-MPO,NaN,NaN


In [7]:
## Adding Waiting data too

In [8]:
wdf = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/E-76-Waiting-list.xlsx', sheet_name=None), ignore_index=True)



In [9]:
wdf.head()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,...,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Warning
0,DISTRICT,BPMP,5904(141),Humboldt County,2021-08-11,NaT,NaT,NaT,NaT,53348.54,...,NaN,NaN,In Humboldt County On Mccann Road Over The Eel...,Bridge Preventive Maintenance,5,NaT,NaT,NON-MPO,2021-11-01,NaN
1,DISTRICT,ATPL,5904(151),Humboldt County,2021-10-22,NaT,NaT,NaT,NaT,1358000.00,...,NaN,NaN,On Sr255 From The Dean Avenue/pacific Avenue I...,Class 1 Multi-use Trail (tc),5,NaT,NaT,NON-MPO,NaT,NaN
2,DISTRICT,BPMP,5904(156),Humboldt County,2021-08-05,NaT,NaT,NaT,NaT,629249.10,...,NaN,NaN,Various Locations In Humboldt County,Bridge Preventive Maintenance: Minor Concrete ...,3,NaT,NaT,NON-MPO,NaT,NaN
3,DISTRICT,BRLO,5904(165),Humboldt County,2021-09-13,NaT,NaT,NaT,NaT,9607.61,...,NaN,NaN,In Humboldt County On Jacoby Creek Bridge At B...,Bridge Rehabilitation Child Project Revegetati...,2,NaT,NaT,NON-MPO,NaT,NaN
4,DISTRICT,HSIPL,5904(181),Humboldt County,2021-10-20,NaT,NaT,NaT,NaT,154662.00,...,NaN,NaN,Intersection Of Redwood Drive And Redway Drive...,Install Enhanced Pedestrian Crossing With Bulb...,3,NaT,NaT,NON-MPO,NaT,NaN


In [10]:
df = pd.concat([odf, wdf], ignore_index=True)

In [11]:
df.head()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,...,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28,Warning
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,...,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,NaN,NaN,NaN
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,...,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,NaN,NaN
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,...,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,NaN,NaN
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,...,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,NaN,NaN
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,...,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,NaN,NaN


In [12]:
df.sample(10)

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,...,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28,Warning
19212,Obligated,BRLO,5930(058),Calaveras County,2021-04-02 00:00:00,2021-04-16 00:00:00,2021-05-11 00:00:00,2021-05-12 00:00:00,2021-05-18 00:00:00,0.00,...,HBPLOCAL,Lime Creek Road (bridge 30c0018) Over Youngs C...,Bridge Replacement (tc),5,2021-03-31 00:00:00,2021-04-16 00:00:00,NON-MPO,NaT,NaN,NaN
16662,Obligated,HSIPL,5328(084),Garden Grove,2020-04-20 00:00:00,2020-04-24 00:00:00,2020-05-01 00:00:00,2020-05-04 00:00:00,2020-05-11 00:00:00,300000.00,...,SCAG015,Intersection Of Trask Avenue And Roxey Drive,New Traffic Signal,2,2020-04-09 00:00:00,2020-04-20 00:00:00,SCAG,NaN,NaN,NaN
5629,Obligated,STEPCR,6049(012),Southern California Association Of Governments,2016-10-14,2016-10-14,2016-10-14,2016-10-14,2016-10-24,-3926.99,...,NaN,Los Angeles County And Other Surrounding Count...,Use Of Invest To Inform The 2016 Rtp/scs,3,NaN,NaN,SCAG,NaN,NaN,NaN
2896,Obligated,BRLO,5183(007),Dunsmuir,2017-12-06,2017-12-07,2017-12-11,2017-12-11,2017-12-15,415000.00,...,230-0000-0031,Butterfly Ave. Bridge #02c-0076 Over The Sacra...,Bridge Replacement (tc),1,2017-11-30,2017-12-05,NON-MPO,NaN,NaN,NaN
9542,Obligated,BRLO,5946(116),Tulare County,2015-02-10,2015-03-13,2015-03-24,2015-03-25,2015-03-29,50000.00,...,TUL11-120,D129 Over Sand Creek (46c0187),Bridge Replace/rehabilitation (tc),2,NaN,NaN,TCAG,NaN,NaN,NaN
18150,Obligated,HSIPL,5927(114),Marin County,2021-01-21 00:00:00,2021-01-21 00:00:00,2021-01-21 00:00:00,2021-01-21 00:00:00,2021-01-25 00:00:00,-7875.61,...,VAR170002,Sir Francis Drake Blvd Mp 8.60-9.53 Marin County,Install Dynamic Variable Speed Warning Signs A...,3,2021-01-14 00:00:00,2021-01-15 00:00:00,MTC,NaT,NaN,NaN
12922,Obligated,CML,5063(150),Santa Ana,2013-11-06,2013-12-04,2014-01-13,2014-01-13,2014-01-14,102844.00,...,ORA113021,Newhope Street From 5th Street To Westminster ...,Class Ii Bikeway,2,NaN,NaN,SCAG,NaN,NaN,NaN
9265,Obligated,BPMPL,5085(034),Merced,2015-04-28,2015-04-28,2015-04-30,2015-05-08,2015-05-12,35412.00,...,CT004,Several Locations (bridges 39c0028 & 39c0041),Bridge Preventive Maintenance,3,NaN,NaN,SCAG,NaN,NaN,NaN
11056,Obligated,STPL,5006(774),Los Angeles,2014-08-01,2014-08-07,2014-08-08,2014-08-13,2014-08-13,11500000.00,...,LA11G1/LA0G911,City Wide In City Of Los Angeles:38 Miles,Fa Resurfacing Xiii :cold Milled/overlaid Hma ...,1,NaN,NaN,SCAG,NaN,NaN,NaN
7226,Obligated,HSIPL,5287(046),Wasco,2016-02-26,2016-02-26,2016-03-02,2016-03-03,2016-03-11,33800.00,...,KER140061,Various Locations,Replace/upgrade Signs (tc),1,NaN,NaN,KCOG,NaN,NaN,NaN


In [13]:
print(len(df))

20447


## Data Cleaning

In [14]:
df.Agency.unique()

array(['Humboldt County', 'Mendocino County', 'Sacramento County',
       'Sutter County', 'Contra Costa County', 'Dinuba', 'Arvin',
       'Fresno', 'Tulare County', 'Palmdale', 'Los Angeles County',
       'Orange County Transportation Authority', 'Yolo County',
       'Nevada County', 'Fresno County', 'Ridgecrest', 'Lassen County',
       'Modoc County', 'Butte County', 'Yuba City', 'Concord',
       'Napa County', 'Pismo Beach', 'Ojai', 'Los Angeles', 'Chino Hills',
       'Indio', 'Moreno Valley', 'Turlock', 'Riverbank', 'Stockton',
       'Chula Vista', 'Sonoma County', 'Santa Clara County',
       'Kings County', 'Long Beach', 'Norwalk', 'Rancho Palos Verdes',
       'Southern California Association of Governments', 'Pasadena',
       'Riverside', 'Carlsbad', 'Santa Barbara County', 'Delano',
       'Merced County', 'South Lake Tahoe', 'Yuba County',
       'Metropolitan Transportation Commission', 'San Mateo',
       'Solano Transportation Authority', 'University Of California'

Some Agencies are all caps, others are not. Applying str.title() to correct this. 

In [15]:
df.Agency = df.Agency.str.title()

In [16]:
df.Agency.unique()

array(['Humboldt County', 'Mendocino County', 'Sacramento County',
       'Sutter County', 'Contra Costa County', 'Dinuba', 'Arvin',
       'Fresno', 'Tulare County', 'Palmdale', 'Los Angeles County',
       'Orange County Transportation Authority', 'Yolo County',
       'Nevada County', 'Fresno County', 'Ridgecrest', 'Lassen County',
       'Modoc County', 'Butte County', 'Yuba City', 'Concord',
       'Napa County', 'Pismo Beach', 'Ojai', 'Los Angeles', 'Chino Hills',
       'Indio', 'Moreno Valley', 'Turlock', 'Riverbank', 'Stockton',
       'Chula Vista', 'Sonoma County', 'Santa Clara County',
       'Kings County', 'Long Beach', 'Norwalk', 'Rancho Palos Verdes',
       'Southern California Association Of Governments', 'Pasadena',
       'Riverside', 'Carlsbad', 'Santa Barbara County', 'Delano',
       'Merced County', 'South Lake Tahoe', 'Yuba County',
       'Metropolitan Transportation Commission', 'San Mateo',
       'Solano Transportation Authority', 'University Of California'

In [17]:
pd.options.display.float_format = "{:.2f}".format
pd.set_option('display.max_columns', None)

In [18]:
df.head()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,Waiting Days,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28,Warning
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904.00,1,E-76 approved on,NaN,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,NaN,NaN,NaN
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910.00,1,E-76 approved on,NaN,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,NaN,NaN
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904.00,1,E-76 approved on,NaN,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,NaN,NaN
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924.00,3,E-76 approved on,NaN,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,NaN,NaN
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924.00,3,E-76 approved on,NaN,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,NaN,NaN


In [19]:
df['Waiting Days'].unique()

array([       nan, 4.3039e+04, 1.0000e+01, 8.8000e+01, 4.9000e+01,
       1.2000e+01, 8.0000e+01, 1.1600e+03, 1.2500e+02, 1.2900e+02,
       1.2400e+02, 6.5400e+02, 1.1500e+02, 5.4000e+01, 1.1800e+02,
       1.7900e+02, 3.4700e+02, 6.3600e+02, 5.5000e+02, 1.1000e+01,
       7.3000e+01, 5.0100e+02, 5.7300e+02, 3.3600e+02, 2.4000e+01,
       7.3300e+02, 3.3500e+02, 6.0000e+00, 6.9000e+01, 2.0800e+02,
       2.5500e+02, 1.5590e+03, 2.8000e+01, 6.1000e+01, 5.9000e+01,
       3.5500e+02, 1.5100e+02, 3.9200e+02, 9.6000e+01, 1.7000e+01,
       5.5000e+01, 1.9600e+02, 1.3000e+02, 4.0600e+02, 4.0500e+02,
       4.0400e+02, 5.8100e+02, 2.1000e+01, 9.1000e+01, 1.3900e+02,
       7.4000e+01, 1.4000e+01, 8.1000e+01, 1.1600e+02, 6.8000e+01,
       6.7000e+01, 1.5800e+02, 1.6000e+02, 5.0000e+00, 1.6100e+02,
       1.4300e+02, 1.0360e+03, 1.5930e+03, 5.2000e+01, 1.5880e+03,
       1.0060e+03, 1.2300e+02, 9.8000e+02, 8.2000e+01, 4.4100e+02,
       2.0420e+03, 7.5000e+01, 4.9500e+02, 5.7900e+02, 7.3900e

In [25]:
df.drop('Waiting Days', axis=1, inplace=True)

In [20]:
df['Today'].unique()

array([nan, '2020-01-01', '2019-06-06 9:00 am', '2021-01-04',
       Timestamp('2021-10-01 00:00:00'), NaT,
       Timestamp('2021-11-01 00:00:00')], dtype=object)

In [21]:
df.drop('Today', axis=1, inplace=True)

In [22]:
df.drop('Unnamed: 28', axis=1, inplace=True)

In [23]:
df.rename(columns={'Project NO': 'project_no',
                        'Fed requested': 'fed_requested',
                        'AC requested': 'ac_requested',
                        'total requested': 'total_requested',
                        'FTIP No': 'ftip_no',
                        'project location': 'project_location',
                        'Location': 'location',
                        'PREFIX': 'prefix',
                        'Agency': 'agency',
                        'prepared date': 'prepared_date',
                        'submit  to HQ date': 'submit_to_hq_date',
                        'To FMIS date': 'to_fmis_date',
                        'Status Comment': 'status_comment',
                        'Locode': 'locode',
                        'DIST': 'dist',
                        'Status': 'status',
                        'dist processing Days': 'dist_processing_days',
                        'HQ processing Days': 'hq_processing_days',
                        'FHWA processing Days': 'fhwa_processing_days',
                        'Type of Work': 'type_of_work',
                        'SEQ': 'seq',
                        'Locode': 'locode',
                        'DIST': 'dist',
                        'Status': 'status',
                        'Date Request Initiated': 'date_request_initiated',
                        'date completed request': 'date_completed_request',
                        'MPO': 'mpo'}, inplace=True)

In [51]:
df.rename(columns={'HQ review date': 'hq_review_date',
                        'submit to FHWA date': 'submit_to_fhwa_date',
                        'Warning': 'warning'}, inplace=True)


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20445 entries, 0 to 20446
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   location                20445 non-null  object 
 1   prefix                  20445 non-null  object 
 2   project_no              20445 non-null  object 
 3   agency                  20445 non-null  object 
 4   prepared_date           20427 non-null  object 
 5   submit_to_hq_date       20270 non-null  object 
 6   hq_review_date          20238 non-null  object 
 7   submit_to_fhwa_date     20220 non-null  object 
 8   to_fmis_date            20220 non-null  object 
 9   fed_requested           20444 non-null  float64
 10  ac_requested            20445 non-null  float64
 11  total_requested         20444 non-null  float64
 12  status_comment          20445 non-null  object 
 13  locode                  18946 non-null  object 
 14  dist                    20445 non-null

In [27]:
df['total_requested'] = df['total_requested'].astype(float)

ValueError: could not convert string to float: '2748.3NA999'

In [30]:
df.loc[df['total_requested']=='2748.3NA999']

,location,prefix,project_no,agency,prepared_date,submit_to_hq_date,HQ review date,submit to FHWA date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,Warning
15547,Obligated,ER,38Y0(001),Los Angeles County,2019-11-12 00:00:00,2019-12-04 00:00:00,2019-12-05 00:00:00,2019-12-06 00:00:00,2019-12-19 00:00:00,1017276.52,-1019159.54,2748.3NA999,Authorized,5953,7,E-76 approved on,28.00,2.00,13.00,NaN,"Mulholland Hwy, Et Al Guardrail Replacement","Install Temporary Traffic Signs, Remove And Re...",2,2019-11-06 00:00:00,2019-11-13 00:00:00,SCAG,NaN


Issue here, and in the following row where 'NA' appears between numbers. Need to determine whether or not to manually override in the exel sheet. 

In [1]:
## removing row with string until we can clarify the true entry.

In [31]:
delete_row = df[df["total_requested"]== '2748.3NA999'].index
df = df.drop(delete_row)

In [32]:
df.loc[df['total_requested']=='2748.3NA999']

,location,prefix,project_no,agency,prepared_date,submit_to_hq_date,HQ review date,submit to FHWA date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,Warning


In [33]:
df['total_requested'] = df['total_requested'].astype(float)

ValueError: could not convert string to float: '30169.98NA99'

In [35]:
df.loc[df['total_requested']=='30169.98NA99']

,location,prefix,project_no,agency,prepared_date,submit_to_hq_date,HQ review date,submit to FHWA date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,Warning
15548,Obligated,ER,38Y0(003),Los Angeles County,2019-11-13 00:00:00,2019-11-14 00:00:00,2019-12-05 00:00:00,2019-12-06 00:00:00,2019-12-20 00:00:00,1033518.07,-1001297.90,30169.98NA99,Authorized,5953,7,E-76 approved on,8.00,22.00,14.00,NaN,Kanan Dume Rd From Mm 8.15 To Mm 11.75,"Install Temporary Traffic Signs, Remove & Repl...",2,2019-11-06 00:00:00,2019-11-13 00:00:00,SCAG,NaN


In [2]:
# removing this row as well until we can verify the total funding.

In [36]:
delete_row = df[df["total_requested"]== '30169.98NA99'].index
df = df.drop(delete_row)

In [37]:
df.loc[df['total_requested']=='30169.98NA99']

,location,prefix,project_no,agency,prepared_date,submit_to_hq_date,HQ review date,submit to FHWA date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,Warning


In [38]:
df['total_requested'] = df['total_requested'].astype(float)

In [40]:
print(df['prepared_date'].dtypes)

object


In [42]:
df['prepared_date'].unique()

array(['2018-12-18', '2018-12-17', '2018-12-07', ...,
       Timestamp('2021-10-06 00:00:00'), Timestamp('2021-09-28 00:00:00'),
       Timestamp('2021-09-21 00:00:00')], dtype=object)

In [49]:
print(df.dtypes)

location                   object
prefix                     object
project_no                 object
agency                     object
prepared_date              object
submit_to_hq_date          object
HQ review date             object
submit to FHWA date        object
to_fmis_date               object
fed_requested             float64
ac_requested              float64
total_requested           float64
status_comment             object
locode                     object
dist                        int64
status                     object
dist_processing_days      float64
hq_processing_days        float64
fhwa_processing_days      float64
ftip_no                    object
project_location           object
type_of_work               object
seq                         int64
date_request_initiated     object
date_completed_request     object
mpo                        object
Warning                    object
dtype: object


In [53]:
cols = ['prepared_date','to_fmis_date','submit_to_fhwa_date','submit_to_hq_date','hq_review_date','date_request_initiated','date_completed_request']
df[cols] = df[cols].applymap(lambda x : pd.to_datetime(x, format = '%Y-%m-%d'))


In [54]:
df.head()

,location,prefix,project_no,agency,prepared_date,submit_to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904.00,1,E-76 approved on,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaT,NaT,NON-MPO,NaN
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910.00,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904.00,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924.00,3,E-76 approved on,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924.00,3,E-76 approved on,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20445 entries, 0 to 20446
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   location                20445 non-null  object        
 1   prefix                  20445 non-null  object        
 2   project_no              20445 non-null  object        
 3   agency                  20445 non-null  object        
 4   prepared_date           20427 non-null  datetime64[ns]
 5   submit_to_hq_date       20270 non-null  datetime64[ns]
 6   hq_review_date          20238 non-null  datetime64[ns]
 7   submit_to_fhwa_date     20220 non-null  datetime64[ns]
 8   to_fmis_date            20220 non-null  datetime64[ns]
 9   fed_requested           20444 non-null  float64       
 10  ac_requested            20445 non-null  float64       
 11  total_requested         20444 non-null  float64       
 12  status_comment          20445 non-null  object

In [56]:
df[["projectID", "projectNO"]] = df["project_no"].str.split(pat="(", expand=True)


In [58]:
df.projectNO = [x.replace(")", "") for x in df.projectNO]

In [59]:
df.head()

,location,prefix,project_no,agency,prepared_date,submit_to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904.00,1,E-76 approved on,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaT,NaT,NON-MPO,NaN,5904,121
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910.00,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,32D0,008
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904.00,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,4820,004
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924.00,3,E-76 approved on,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,5924,244
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924.00,3,E-76 approved on,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,5924,214


## Locode and Project ID Columns 

Hypothesis:
    Project IDs and Locodes, or unique agency identifiers, are the same 
    
As it stands now, these columns have a mix of letters and numbers, making it hard to compare the two. 
In this block we will work to convert these columns, while trying to not convert the non-numeric numbers. 

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20445 entries, 0 to 20446
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   location                20445 non-null  object        
 1   prefix                  20445 non-null  object        
 2   project_no              20445 non-null  object        
 3   agency                  20445 non-null  object        
 4   prepared_date           20427 non-null  datetime64[ns]
 5   submit_to_hq_date       20270 non-null  datetime64[ns]
 6   hq_review_date          20238 non-null  datetime64[ns]
 7   submit_to_fhwa_date     20220 non-null  datetime64[ns]
 8   to_fmis_date            20220 non-null  datetime64[ns]
 9   fed_requested           20444 non-null  float64       
 10  ac_requested            20445 non-null  float64       
 11  total_requested         20444 non-null  float64       
 12  status_comment          20445 non-null  object

In [61]:
df.sample(10)

,location,prefix,project_no,agency,prepared_date,submit_to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO
671,Obligated,STPL,5063(128),Santa Ana,2018-08-28,2018-08-28,2018-08-28,2018-08-28,2018-09-10,0.00,0.00,225562.12,Authorized,5063.00,12,E-76 approved on,NaN,0.00,13.00,ORA120520,Grand Avenue From 1st Street To 4th Street,Roadway Widening,4,NaT,NaT,SCAG,NaN,5063,128
6307,Obligated,SRTSL,5936(102),Santa Cruz County,2016-07-05,2016-07-14,2016-07-14,2016-07-14,2016-07-20,100000.00,0.00,144224.00,Authorized,5936.00,5,E-76 approved on Prepared on 10/6/2015,9.00,0.00,6.00,BP01GP,"Bush St,hermon St Between Oak And Bush, Laurel...",To Improve Routes To Boulder Creek Elementary ...,2,NaT,NaT,AMBAG,NaN,5936,102
11461,Obligated,BPMPL,5936(119),Santa Cruz County,2014-06-30,2014-07-02,2014-07-07,2014-07-07,2014-07-09,22133.00,0.00,25000.00,Authorized,5936.00,5,E-76 approved on E-76 approved on,2.00,5.00,2.00,GP HBRR,Zayante Rd At Zayante Crk 36c-0142,Concrete Spall Repair,1,NaT,NaT,AMBAG,NaN,5936,119
335,Obligated,CML,5055(178),Anaheim,2018-10-17,2018-10-17,2018-10-17,2018-10-29,2018-11-02,-136933.09,0.00,-139580.73,Authorized,5055.00,12,E-76 approved on,NaN,12.00,4.00,ORA113001,Ball Road From Magnolia Avenue To The Artic On...,Class Ii Bicycle Lanes And Class Iii Bicycle S...,3,NaT,NaT,SCAG,NaN,5055,178
19740,Obligated,BRLS,5954(168),San Bernardino County,2021-08-09,2021-08-10,2021-08-11,2021-08-16,2021-08-27,0.00,504000.00,630000.00,Authorized,5954,8,E-76 approved on,1.00,6.00,11.00,SBDLS08,National Trails Highway (route 66) At Kalmia D...,Replace Two Lane Timber Bridge With Two Lane B...,1,2021-08-09,2021-08-09,SCAG,NaN,5954,168
17100,FTA Transferred,FTASTPL,7503(001),San Diego Metropolitan Tranit System,2020-07-03,2020-07-03,2020-07-03,2020-07-31,2020-08-03,0.12,0.00,0.12,Prog Code Q230,7503,11,FTA transferred on 8/3/2020,NaN,28.00,3.00,NaN,NaN,FTA Transfer,1,NaT,NaT,NON-MPO,NaN,7503,001
948,Obligated,ER,32L0(486),Sierra County,2018-03-19,2018-03-27,2018-06-29,2018-07-12,2018-07-19,17706.00,0.00,20000.00,Authorized,5913.00,3,E-76 approved on,15.00,107.00,7.00,NaN,Mountain House Rd 3,New Engineered Embankment System Will Need To ...,1,2018-03-12,2018-03-12,NON-MPO,NaN,32L0,486
8289,Obligated,STPLZ,5277(027),Fairfax,2015-08-21,2015-08-21,2015-09-09,2015-09-10,2015-09-10,95000.00,0.00,95000.00,Authorized,5277.00,4,E-76 approved on E-76 approved on,0.00,20.00,0.00,VAR110045,Creek Road Over Fairfax Creek In The Town Of F...,Seismic Retrofit (tc),3,NaT,NaT,MTC,NaN,5277,027
2516,Obligated,ER,32L0(100),El Dorado County,2018-01-19,2018-01-19,2018-01-23,2018-01-23,2018-02-08,594700.27,-594700.27,0.00,Authorized,5925.00,3,E-76 approved on,-1.00,4.00,16.00,NaN,Elks Club Drive From Us50 To Boca Raton Drive.,Emergency Opening And Restoration Of Sinkhole ...,3,2018-01-20,2018-01-20,SACOG,NaN,32L0,100
5564,Obligated,CML,5095(019),Rocklin,2016-10-26,2016-10-26,2016-10-26,2016-10-26,2016-10-31,-184184.33,0.00,-158449.32,Authorized,5095.00,3,E-76 approved on Prepared on 10/6/2015,0.00,0.00,5.00,PLA25552,North Side Of Pacific St Btw Del Mar Ave To Th...,Rdway Imrpovements Including Bike Lanes,4,NaT,NaT,SACOG,NaN,5095,019


In [51]:
## dataset has a column NBIL, 32L0, 40A0, 5.92E+03, 38Y0, 38R0

## for now, leaving the errors. 

In [62]:
def get_num(x):
    try:
        return int(x)
    except Exception:
        try:
            return float(x)
        except Exception:
            return x

        
# https://stackoverflow.com/questions/56474599/how-to-ignore-errors-in-pandas-to-numeric-and-change-str-to-int

In [63]:
df['locode'] = df['locode'].apply(get_num)

In [64]:
df['projectID'] = df['projectID'].apply(get_num)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20445 entries, 0 to 20446
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   location                20445 non-null  object        
 1   prefix                  20445 non-null  object        
 2   project_no              20445 non-null  object        
 3   agency                  20445 non-null  object        
 4   prepared_date           20427 non-null  datetime64[ns]
 5   submit_to_hq_date       20270 non-null  datetime64[ns]
 6   hq_review_date          20238 non-null  datetime64[ns]
 7   submit_to_fhwa_date     20220 non-null  datetime64[ns]
 8   to_fmis_date            20220 non-null  datetime64[ns]
 9   fed_requested           20444 non-null  float64       
 10  ac_requested            20445 non-null  float64       
 11  total_requested         20444 non-null  float64       
 12  status_comment          20445 non-null  object

In [66]:
compare_col = np.where(df["locode"] == df["projectID"], True, False)
df["compare_id_locode"] = compare_col
df.head()

,location,prefix,project_no,agency,prepared_date,submit_to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904,1,E-76 approved on,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaT,NaT,NON-MPO,NaN,5904,121,True
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,32D0,008,False
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,4820,004,False
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924,3,E-76 approved on,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,5924,244,True
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924,3,E-76 approved on,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,5924,214,True


In [67]:
df['compare_id_locode'].value_counts()

True     16677
False     3768
Name: compare_id_locode, dtype: int64

In [68]:
group = df.groupby('agency')
crosswalk_all = group.apply(lambda x: x['locode'].unique())
print(crosswalk_all)

agency
Access Services                                             [6312]
Agoura Hills                                           [5435, nan]
Ala-Con Costa T                                             [6002]
Alameda                                                     [5014]
Alameda - Contra Costa Transit District                     [6002]
                                                    ...           
Yrts                                                         [nan]
Yuba City                                                   [5163]
Yuba County                                                 [5916]
Yucaipa                                    [5457, nan, 5954, NBIL]
Yucca Valley                                           [5466, nan]
Length: 671, dtype: object


In [69]:
#crosswalk_all.to_csv('crosswalk_all.csv')

In [70]:
#https://stackoverflow.com/questions/29530232/how-to-check-if-any-value-is-nan-in-a-pandas-dataframe

nan_rows = df[df['locode'].isnull()]

In [71]:
nan_rows.sample(10)

,location,prefix,project_no,agency,prepared_date,submit_to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode
12931,Obligated,STPL,5108(142),Long Beach,2013-12-24,2014-01-07,2014-01-07,2014-01-10,2014-01-10,-97576.00,0.00,-99966.00,Authorized,NaN,7,E-76 approved on E-76 approved on,14.00,3.00,0.00,LA11G3,68th St: Union Pacific Rail Road To Paramount ...,Road Rehabilitation. (tc),5,NaT,NaT,SCAG,NaN,5108,142,False
11367,Obligated,STPL,5929(244),San Joaquin,2014-07-01,2014-07-01,2014-07-10,2014-07-10,2014-07-15,-273000.00,0.00,-273000.00,Authorized,NaN,10,E-76 approved on E-76 approved on,0.00,9.00,5.00,SJ07-3071,"Lower Sacramento(woodbridge Rd To Jahant Rd), ...",Re-surface Existing Roadway (tc),3,NaT,NaT,STANCOG,NaN,5929,244,False
10805,Obligated,CML,5929(215),San Joaquin,2014-09-02,2014-09-12,2014-09-12,2014-09-12,2014-09-16,0.00,0.00,73165.41,Authorized,NaN,10,E-76 approved on E-76 approved on,10.00,0.00,4.00,212-0000-0373,South Stockton Sidewalks,"Curb,gutter,sidewalks,.adj Rdwy To Match Gutter",3,NaT,NaT,KCOG,NaN,5929,215,False
12577,Obligated,BRLO,5929(235),San Joaquin,2014-03-04,2014-03-04,2014-03-07,2014-03-07,2014-03-10,165533.00,0.00,165533.00,Authorized,NaN,10,E-76 approved on E-76 approved on,0.00,3.00,3.00,SJ07-3002,Wimer Road Over Nb Indian Creek (br 29c0303),Bridge Replacement (tc),2,NaT,NaT,SCAG,NaN,5929,235,False
10337,Obligated,DEM05L,5063(145),Santa Ana,2014-12-16,2014-12-16,2014-12-16,2014-12-16,2014-12-17,0.00,0.00,0.00,Authorized,NaN,12,E-76 approved on E-76 approved on,0.00,0.00,1.00,ORA125,Bristol Street From Warner Avenue To St. Andre...,Roadway Widening,3,NaT,NaT,SCAG,NaN,5063,145,False
10602,Obligated,HSIPL,5385(042),Ridgecrest,2014-10-15,2014-10-23,2014-10-24,2014-10-24,2014-10-24,-1683.00,0.00,-1870.00,Authorized,NaN,9,E-76 approved on E-76 approved on,8.00,1.00,0.00,KER110601,Intersection Of S. China Lake Blvd. & Upjohn A...,Signals And Signal Interconnect,4,NaT,NaT,SCAG,NaN,5385,042,False
11673,Obligated,SRTSLNI,6445(002),Chula Vista Elementary School District,2014-06-19,2014-06-19,2014-06-19,2014-06-19,2014-06-20,-3099.98,0.00,-3099.98,Authorized,NaN,11,E-76 approved on E-76 approved on,0.00,0.00,1.00,NaN,Various Locations Throughout Chula Vista,Non-infrastructure Program,2,NaT,NaT,SJCOG,NaN,6445,002,False
8716,Obligated,CML,6305(011),Yrts,2015-07-09,2015-07-13,2015-07-15,2015-07-17,2015-07-17,39283.00,0.00,39283.00,Authorized,NaN,10,E-76 approved on E-76 approved on,4.00,4.00,0.00,YARTS07,"Merced, Mariposa, Mono Counties",Yarts Public Outreach And Marketing (tc),1,NaT,NaT,ER NONE,NaN,6305,011,False
12750,Obligated,STPL,5428(009),Poway,2014-02-19,2014-02-19,2014-02-19,2014-02-19,2014-02-19,-211.78,0.00,527161.35,Authorized,NaN,11,E-76 approved on E-76 approved on,0.00,0.00,0.00,NaN,Espola Rd. From 900' S/o Twin Peaks To Titan,Roadway Widening,2,NaT,NaT,SANDAG,NaN,5428,009,False
13006,Obligated,TCSPL,5450(072),Santa Clarita,2013-12-16,2013-12-16,2013-12-27,2014-01-02,2014-01-02,0.00,0.00,-14436.00,Authorized,NaN,7,E-76 approved on E-76 approved on,0.00,17.00,0.00,LA0G734,Mcbean Parkway And Avenida Navaree.,Construct Bus Turn-out And Left Turn Pocket.,3,NaT,NaT,SCAG,NaN,5450,072,False


In [72]:
print(len(nan_rows))

1499


In [63]:
#checking to see if these NaN values are also in the downloadable data. 

In [64]:
# There are the same number of NaNs (1499) on the excel sheet. 
# So without these, there are still 2,206 

## Comparing columns and Summarizing Data

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20445 entries, 0 to 20446
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   location                20445 non-null  object        
 1   prefix                  20445 non-null  object        
 2   project_no              20445 non-null  object        
 3   agency                  20445 non-null  object        
 4   prepared_date           20427 non-null  datetime64[ns]
 5   submit_to_hq_date       20270 non-null  datetime64[ns]
 6   hq_review_date          20238 non-null  datetime64[ns]
 7   submit_to_fhwa_date     20220 non-null  datetime64[ns]
 8   to_fmis_date            20220 non-null  datetime64[ns]
 9   fed_requested           20444 non-null  float64       
 10  ac_requested            20445 non-null  float64       
 11  total_requested         20444 non-null  float64       
 12  status_comment          20445 non-null  object

In [74]:
df.sample(10)

,location,prefix,project_no,agency,prepared_date,submit_to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode
15056,Obligated,HSIPL,5932(089),Tuolumne County,2019-10-30,2019-10-30,2019-10-30,2019-10-30,2019-10-31,-0.02,0.00,0.00,Authorized,5932,10,E-76 approved on,6.00,0.00,1.00,230-0000-0159,Tuolumne County- Various Locations,Retrofit Regulatory And Warning Signs,3,2019-10-24,2019-10-25,NON-MPO,NaN,5932,089,True
3828,Obligated,SRTSL,5080(029),Lompoc,2017-07-26,2017-07-27,2017-07-27,2017-07-27,2017-08-01,0.00,0.00,5.41,Authorized,5080,5,E-76 approved on,3.00,0.00,5.00,VAR03,City Of Lompoc Various Schools,"Sidewalks, Curb Ramps And Bulb-outs",4,2017-07-24,2017-07-25,SBCG,NaN,5080,029,True
9872,Obligated,STPL,5184(017),Tehachapi,2015-02-13,2015-02-13,2015-02-13,2015-02-13,2015-02-15,-0.55,0.00,-0.06,Authorized,NaN,9,E-76 approved on E-76 approved on,0.00,0.00,2.00,KER100408,"Tehachapi Boulevard From Approx. 1,000 Ft West...",Pavement Rehabilitation,3,NaT,NaT,SCAG,NaN,5184,017,False
18074,DISTRICT,FTAFBP,6365(007),Wta,NaT,NaT,NaT,NaT,NaT,1356699.00,0.00,1356699.00,prog code Z240,6365,4,FTA transferred waiting at DISTRICT,NaN,0.00,0.00,NaN,FTA transfer,Construct Additional Ferry Docking/berthing,1,NaT,NaT,MTC,NaN,6365,007,True
12724,Obligated,HSIPL,5419(025),Lancaster,2014-02-20,2014-02-20,2014-02-20,2014-02-20,2014-02-20,-0.01,0.00,0.00,Authorized,NaN,7,E-76 approved on E-76 approved on,0.00,0.00,0.00,NaN,Intersection: Ave. I And 40th St. West .,Construct Left-turn Lane,4,NaT,NaT,SANDAG,NaN,5419,025,False
16393,Obligated,STPL,5281(025),Shafter,2020-04-06,2020-04-06,2020-04-07,2020-04-07,2020-04-14,0.00,0.00,0.00,Authorized,5281,6,E-76 approved on,4.00,1.00,7.00,KER160402,Beech Ave. Between State Ave. And E. Tulare Ave.,"Reconstruct Existing Pavement, Install Curb An...",3,2020-04-02,2020-04-03,KCOG,NaN,5281,025,True
14860,Obligated,BHLS,5004(049),San Diego,2019-05-10,2019-09-06,2019-09-07,2019-09-07,2019-09-12,14434221.00,-3011141.00,14464861.00,Authorized,5004,11,E-76 approved on,120.00,1.00,5.00,NaN,W Mission Bay Drive/san Diego River,Bridge Replacement (57c-0023),9,2019-05-09,2019-06-06,SDAG,NaN,5004,049,True
12032,Obligated,HSIPL,5399(023),California City,2014-04-21,2014-04-23,2014-04-24,2014-04-24,2014-04-28,24750.00,0.00,27500.00,Authorized,NaN,9,E-76 approved on E-76 approved on,2.00,1.00,4.00,KER140601,California City Boulevard From Baron Boulevard...,"Safety Improvements, Install Reflectors, Rumbl...",1,NaT,NaT,SJCOG,NaN,5399,023,False
19103,Obligated,BRLO,5946(189),Tulare County,2019-09-19,2021-04-13,2021-04-15,2021-05-10,2021-05-11,137500.00,0.00,137500.00,Authorized,5946,6,E-76 approved on,8.00,27.00,1.00,TUL11-120,Bridge 46c0195 On Road M348 Over South Fork Of...,Revegetation Required By Nepa (tc),1,2021-04-05,2021-04-13,TCAG,NaN,5946,189,True
1991,Obligated,CML,5109(221),Bakersfield,2018-04-04,2018-04-04,2018-04-04,2018-04-04,2018-04-11,-58150.48,0.00,-65035.61,Authorized,5109,6,E-76 approved on,NaN,0.00,7.00,KER140508,Mohawk St. Between California Ave. And Truxtun...,Construct Raised Median Island,2,NaT,NaT,KCOG,NaN,5109,221,True


In [75]:
from siuba.data import cars_sql
from siuba import group_by, mutate, show_query

In [76]:
df[_.compare_id_locode == False]

,location,prefix,project_no,agency,prepared_date,submit_to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,32D0,008,False
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,4820,004,False
21,Obligated,ACSTP,32L0(510),Butte County,2018-12-17,2018-12-20,2018-12-20,2018-12-20,2018-12-21,0.00,2986207.00,3367920.00,Authorized,5912,3,E-76 approved on,43.00,0.00,1.00,NaN,Oroville Quincy Hwy: Location #1 100 Yards Ab...,Emergency Opening And Restoration Of Roadway S...,1,2018-11-07,2018-11-07,BCAG,NaN,32L0,510,False
24,Obligated,ER,26E0(001),Napa County,2018-12-18,2018-12-18,2018-12-20,2018-12-20,2018-12-21,941474.00,0.00,1026469.00,Authorized,5921,4,E-76 approved on,13.00,2.00,1.00,NaN,Daf 1: Oak Knoll Ave (mpm 0.20); Daf 2: Oak Kn...,Napa County Earthquake Repair - Pavement Resto...,4,2018-11-30,2018-12-05,MTC,NaN,26.00,001,False
54,Obligated,ER,29P0(001),Carlsbad,2018-12-13,2018-12-13,2018-12-13,2018-12-14,2018-12-18,-61058.59,0.00,-53619.67,Authorized,5308,11,E-76 approved on,NaN,1.00,4.00,NaN,"Carlsbad Boulevard, North Of Las Encinas Creek",Emergency Revetment,2,NaT,NaT,SDAG,NaN,29P0,001,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20337,Obligated,ER,32L0(440),Monterey County,2021-10-18,2021-10-18,2021-10-18,2021-10-18,2021-10-22,-232125.67,0.00,-255334.09,Authorized,5944,5,E-76 approved on,4.00,0.00,4.00,NaN,Hall Road At Pini Road,Eo- County Placed Barricades. Contractor Plac...,6,2021-10-14,2021-10-15,AMBAG,NaN,32L0,440,False
20361,Obligated,ER,32L0(560),Los Angeles County,2021-10-18,2021-10-18,2021-10-18,2021-10-18,2021-10-26,-5796.69,0.00,-6547.71,Authorized,5953,7,E-76 approved on,4.00,0.00,8.00,NaN,"The Old Road @ Mm12.62, 13.66 And @ Weldon Rd;...","Eo: Removed Debris Along The Roadway, Shoulder...",2,2021-10-14,2021-10-15,SCAG,NaN,32L0,560,False
20366,Obligated,ER,30X0(001),Los Angeles County,2021-10-18,2021-10-18,2021-10-18,2021-10-18,2021-10-22,0.00,0.00,-195.52,Authorized,5953,7,E-76 approved on,4.00,0.00,4.00,NaN,Soledad Canyon Rd From Mm 15.48 To Mm 18.46,Drainage Structures Installation,3,2021-10-14,2021-10-15,SCAG,NaN,30X0,001,False
20369,Obligated,ER,32L0(554),Trinity County,2021-10-13,2021-10-13,2021-10-13,2021-10-13,2021-10-19,-243.30,0.00,-274.82,Authorized,5953,7,E-76 approved on,6.00,0.00,6.00,NaN,Kanan Dume Rd From Mulholland Hwy To City Mal...,Eo: Debris Removal,3,2021-10-07,2021-10-08,SCAG,NaN,32L0,554,False


In [78]:
df['status_comment'].value_counts()

Authorized                                                         19642
..                                                                   194
Prog Code Z400                                                       147
Prog Code M400                                                        73
Prog Code Z230                                                        60
                                                                   ...  
ON HOLD: INSUFFICIENT SACOG CMAQ APPORTIONMENT BALANCE                 1
HOLD - REQ FOR OBLIGATION OF FLEX STBGP FROM MTC LOAN TO BRIDGE        1
MISSING REASON FOR THE DELAY                                           1
EXPIRED QAP                                                            1
prog code Z300                                                         1
Name: status_comment, Length: 81, dtype: int64

In [79]:
df['location'].value_counts()

Obligated          19642
FTA Transferred      543
DISTRICT             170
HQ                    53
FMIS                  37
Name: location, dtype: int64

In [80]:
df['type_of_work'].nunique()

6882

In [81]:
df['prefix'].nunique()

392

In [82]:
prefix = df['prefix'].value_counts()

In [83]:
prefix.head()

CML      3334
HSIPL    3331
STPL     2810
ER       1607
BRLO     1416
Name: prefix, dtype: int64

In [84]:
print((prefix == 1).sum())

113


In [85]:
prefix.sort_index(inplace=True)
prefix

6140000      2
ACNHPI       2
ACST-ER    238
ACSTP      317
ASCTP        1
          ... 
VPPL        19
VPPP         3
VPPPL        5
VPPTCS       1
WZDXL20      1
Name: prefix, Length: 392, dtype: int64

In [86]:
#Getting a CSV to create a detailed crosswalk with defintions. 
#prefix.to_csv('count_grant_types.csv')

## Convert to CSV for future analysis 

In [88]:
#df.to_csv('clean_obligated_waiting.csv')